# Get packages and data

In [1]:
import sqlalchemy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import yaml

import macrosynergy.management as msm
import macrosynergy.panel as msp
import macrosynergy.signal as mss

## import qstools.indicators as qsi

import warnings
warnings.simplefilter('ignore')

import macrosynergy.management as msm
with open("..//data//config.yml", 'r') as f:
    cf = yaml.load(f, Loader=yaml.FullLoader)

with msm.DataQueryInterface(
            username=cf["dq"]["username"],
            password=cf["dq"]["password"],
            crt="..//data//api_macrosynergy_com.crt",
            key="..//data//api_macrosynergy_com.key") as dq:
    print("Connected: ", dq.check_connection())
    r = dq.get_ts_expression(expression="DB(JPMAQS,GBP_FXXR_NSA,value)", start_date = "2000-01-01")

In [2]:
def dq_single_ticker(tick, metric='value', path = '..//data//'):
    
    """
    Returns standardized dataframe of single ticker
    
    
    :param <str> tick: JPMaQS ticker of form <cid>_<xcat>
    :param <str> metric: metric that is requested, must be 'value', 'eop_lag', 'mop_lag', or 'grade'
    :param <str> path: relative path from notebook to credential files
    
    :return <pd.Dataframe> standardized dataframe with columns 'cid', 'xcats', 'real_date' and 'value.
    """
    
    with open(f"{path}config.yml", 'r') as f:
        cf = yaml.load(f, Loader=yaml.FullLoader)

    with msm.DataQueryInterface(
                username=cf["dq"]["username"],
                password=cf["dq"]["password"],
                crt=f"{path}api_macrosynergy_com.crt",
                key=f"{path}api_macrosynergy_com.key") as dq:
        print("Connected: ", dq.check_connection())
        r = dq.get_ts_expression(expression=f"DB(JPMAQS,{tick},{metric})", start_date = "2000-01-01")
        
    attributes = r[0]['attributes'][0]
    ar_ts = np.array(attributes['time-series'])

    ticker = attributes['expression'].split(',')[1]
    cid, xcat = ticker.split('_', maxsplit=1)  # split string only at first underscore

    ar_cid = np.repeat(cid, repeats = time_series.shape[0])
    ar_xcat = np.repeat(xcat, repeats = time_series.shape[0])

    ar_all = np.column_stack((ar_cid, ar_xcat, ar_ts))
    columns = ['cid', 'xcat', 'real_date', 'value']
    df = pd.DataFrame(data = ar_all, columns = columns)

    df['real_date'] = pd.to_datetime(df['real_date'], yearfirst = True)
    df = df[df['real_date'].dt.dayofweek < 5]  # remove Sats and Suns
    df = df.fillna(value=np.nan)  # replace Nones
    
    return df

In [3]:
df_i = dq_single_ticker('GBP_FXXR_NSA', metric='value', path = '..//data//')

FileNotFoundError: [Errno 2] No such file or directory: '..//data//config.yml'

In [77]:
df_i

,cid,xcat,real_date,value
2,GBP,FXXR_NSA,2000-01-03,0.982100
3,GBP,FXXR_NSA,2000-01-04,-0.639361
4,GBP,FXXR_NSA,2000-01-05,0.281824
5,GBP,FXXR_NSA,2000-01-06,0.284847
6,GBP,FXXR_NSA,2000-01-07,-0.318615
...,...,...,...,...
7950,GBP,FXXR_NSA,2021-10-07,0.321967
7951,GBP,FXXR_NSA,2021-10-08,0.053512
7954,GBP,FXXR_NSA,2021-10-11,-0.079867
7955,GBP,FXXR_NSA,2021-10-12,-0.085202
